# SETUP
- - -

In [1]:
import numpy as np
import tensorflow as tf

### softmax function

In [9]:
x_data = np.array([[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], 
                   [4, 1, 5, 5], [1, 7, 5, 5], [1, 2, 5, 6], 
                   [1, 6, 6, 6], [1, 7, 7, 7]])

y_data = np.array([[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], 
                   [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]])

X = tf.placeholder('float', [None, 4])
Y = tf.placeholder('float', [None, 3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes], name='bias'))

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

In [10]:
x_data.shape, y_data.shape, W.shape, b.shape

((8, 4),
 (8, 3),
 TensorShape([Dimension(4), Dimension(3)]),
 TensorShape([Dimension(3)]))

In [15]:
# 손실 함수, 최적화 알고리즘 정의

cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cross_entropy)

# 모델링

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2001):
    sess.run(train_step, feed_dict={X: x_data, Y: y_data})
    if step % 200 == 0:
        print(step, sess.run(cross_entropy, feed_dict={X: x_data, Y: y_data}))
            
a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
print(a, sess.run(tf.arg_max(a, 1)))

sess.close()

0 4.797593
200 0.57750267
400 0.4704681
600 0.3745716
800 0.2835989
1000 0.23747563
1200 0.2149789
1400 0.19629832
1600 0.18052256
1800 0.1670222
2000 0.15534174
[[7.7862656e-03 9.9220616e-01 7.6045153e-06]] [1]


### animal classification

In [17]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)

In [22]:
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [24]:
xy.shape, x_data.shape, y_data.shape

((101, 17), (101, 16), (101, 1))

In [25]:
nb_classes = 7

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])
Y_one_hot = tf.one_hot(Y, nb_classes)

In [29]:
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

logits = tf.matmul(X, W) + b
y_ = tf.nn.softmax(logits)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cross_entropy)

train_step = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(y_, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [31]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2000):
    sess.run(train_step, feed_dict={X: x_data, Y: y_data})

    if step % 100 == 0:
        loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data})
        print("Step: {:5} | Loss: {:.3f} | Acc: {:.2%}".format(step, loss, acc))

Step:     0 | Loss: 6.280 | Acc: 15.84%
Step:   100 | Loss: 0.841 | Acc: 77.23%
Step:   200 | Loss: 0.484 | Acc: 85.15%
Step:   300 | Loss: 0.315 | Acc: 90.10%
Step:   400 | Loss: 0.231 | Acc: 96.04%
Step:   500 | Loss: 0.186 | Acc: 97.03%
Step:   600 | Loss: 0.156 | Acc: 99.01%
Step:   700 | Loss: 0.136 | Acc: 99.01%
Step:   800 | Loss: 0.120 | Acc: 99.01%
Step:   900 | Loss: 0.107 | Acc: 99.01%
Step:  1000 | Loss: 0.097 | Acc: 100.00%
Step:  1100 | Loss: 0.089 | Acc: 100.00%
Step:  1200 | Loss: 0.082 | Acc: 100.00%
Step:  1300 | Loss: 0.076 | Acc: 100.00%
Step:  1400 | Loss: 0.071 | Acc: 100.00%
Step:  1500 | Loss: 0.066 | Acc: 100.00%
Step:  1600 | Loss: 0.062 | Acc: 100.00%
Step:  1700 | Loss: 0.059 | Acc: 100.00%
Step:  1800 | Loss: 0.056 | Acc: 100.00%
Step:  1900 | Loss: 0.053 | Acc: 100.00%


In [32]:
pred = sess.run(prediction, feed_dict={X: x_data})

for p, y in zip(pred, y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p==int(y), p, int(y)))

[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 3 True Y: 3
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 5 True Y: 5
[True] Prediction: 4 True Y: 4
[True] Prediction: 4 True Y: 4
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 5 True Y: 5
[True] Prediction: 0 True Y: 0
[True] P